In [ ]:
#!pip install openpyxl # Uncomment to install openpyxl

# Instructions
1. Run `Libraries`.
1. Run `Optimisation Algorithm` (Top to Bottom).
1. Make sure `MiniModels.ipynb`, `PeriodMaximisation.py` and `Battery.py` are in the same directory with this notebook.
1. Go to `User Usage` Section, put the `input` and `output` filepath including the formats.
1. Go to respective sections, run and save to respective formats (Top to Bottom).

### Libraries

In [1]:
import pandas as pd
import numpy as np
import time

import PeriodMaximisation as pm
from Battery import Battery

%run ./MiniModels.ipynb

## Optimisation Algorithm

In [2]:
def ConvertFormat_BunToMeat(df):
    
    # Converting simple format to actual format
    df["Power"] = np.where(df["Status"] == "Charge", df["Actual"] * 2, df["Actual"] * 2 / 0.9)
    df = pm.renameColumns(df[["Time", "Price", "Power", "Actual", "Opening Capacity", "Closing Capacity"]])
    
    return df

def ConvertFormat_MeatToBun(act, df):
    
    # Converting actual format to simple format
    act = act[["Time (UTC+10)", "Regions VIC Trading Price ($/MWh)", 
               "Market Dispatch (MWh)", "Opening Capacity (MWh)", 
               "Closing Capacity (MWh)"]] # Select time and victoria prices
    act.columns = ["Time", "Price", "Actual", "Opening Capacity", "Closing Capacity"] # Rename columns
    act["Time"] = pd.to_datetime(df["Time"]) # Convert data type
    act = act.sort_values("Time").reset_index(drop = True) # Finalise
    act["Restrict"] = - np.where(act["Actual"] <= 0, act["Actual"] * 0.9, act["Actual"] / 0.9)
    act["Status"] = np.where(act["Actual"] < 0, "Charge", np.where(act["Actual"] > 0, "Discharge", "Do Nothing"))

    sim = pd.merge(df["Time"], act.drop("Time", axis = 1), left_index = True, right_index = True)

    return sim

In [7]:
def TopBun(df, n = 17):
    Model = MovingAverage(n, df) # Moving Average
    Model = Maximisation(Model, df) # Region Maximisation
    Model = LossRemoval(Model) # Loss Removal
    Model = Stationary(Model, df) # Stationary Maximisation
    Model = ShiftAction(Model, df) # Action Shift
    
    return Model

def Meat(df):
    tmp = df.copy()
    # Converting format
    tmp = ConvertFormat_BunToMeat(tmp)
    
    # Get time period
    time = pm.getTimePeriod(tmp)
    
    # Run Dependendy Optimisation Algorithm
    tmp = pm.DependencyOptimisation(tmp, time) # PeriodOptimisation
    
    # Convert back format
    tmp = ConvertFormat_MeatToBun(tmp, df)
    
    return tmp

def LowerBun(df):
    
    Model = Maximisation(df, df = df) # Region Maximisation
    Model = Stationary(Model, df = df) # Stationary Maximisation
    Model = ShiftAction(Model, df = df) # Action Shift

    return Model

def SandwichModel(original_data):
    tmp = original_data.copy()
    
    start = time.time()
    print("Calculating First Step")
    df = TopBun(tmp, n = 17)
    first = time.time()
    print(f"It took {first - start} seconds for First Step")
    
    print("#######################\n")
    
    print("Calculating Second Step")
    df = Meat(df)
    second = time.time()
    print(f"It took {second - first} seconds for Second Step")
    
    print("#######################\n")
    
    print("Calculating Final Step")
    #df = LowerBun(df)
    third = time.time()
    print(f"It took {third - second} seconds for Third Step")
    total = time.time()
    
    print("#######################\n")
    
    print(f"In total cooking time took {total - start} seconds")
    df = ConvertFormat_BunToMeat(df)
    return df

### All Data

In [4]:
df = pd.read_excel("../../preprocessed_data/original_data.xlsx") # Open original data
df = df[["Time (UTC+10)", "Regions VIC Trading Price ($/MWh)"]] # Select time and victoria prices
df.columns = ["Time", "Price"] # Rename columns
df["Time"] = pd.to_datetime(df["Time"]) # Convert data type
df = df.sort_values("Time").reset_index(drop = True) # Finalise

KeyboardInterrupt: 

In [ ]:
final_result = SandwichModel(df)

print(f"Total Revenue for this dataset is : {pm.computeRevenue(final_result)}")

### Training Set

In [ ]:
train_df = pd.read_excel("../../preprocessed_data/training_data.xlsx") # Open original data
train_df = train_df[["Time (UTC+10)", "Regions VIC Trading Price ($/MWh)"]] # Select time and victoria prices
train_df.columns = ["Time", "Price"] # Rename columns
train_df["Time"] = pd.to_datetime(train_df["Time"]) # Convert data type
train_df = train_df.sort_values("Time").reset_index(drop = True) # Finalise

In [ ]:
final_training_result = SandwichModel(train_df)

print(f"Total Revenue for this dataset is : {pm.computeRevenue(final_training_result)}")

### Testing Set

In [5]:
test_df = pd.read_excel("../../preprocessed_data/testing_data.xlsx") # Open original data
test_df = test_df[["Time (UTC+10)", "Regions VIC Trading Price ($/MWh)"]] # Select time and victoria prices
test_df.columns = ["Time", "Price"] # Rename columns
test_df["Time"] = pd.to_datetime(test_df["Time"]) # Convert data type
test_df = test_df.sort_values("Time").reset_index(drop = True) # Finalise

In [6]:
final_testing_result = SandwichModel(test_df)

print(f"Total Revenue for this dataset is : {pm.computeRevenue(final_testing_result)}")

Calculating First Step
It took 4.222799301147461 seconds for First Step
#######################

Calculating Second Step
           Time (UTC+10)  Regions VIC Trading Price ($/MWh)  Raw Power (MW)  \
0    2021-07-01 00:30:00                              90.51            -0.0   
1    2021-07-01 01:00:00                              73.91            -0.0   
2    2021-07-01 01:30:00                              33.79          -300.0   
3    2021-07-01 02:00:00                              43.57            -0.0   
4    2021-07-01 02:30:00                              45.90           270.0   
...                  ...                                ...             ...   
2155 2021-08-14 22:00:00                              49.93            -0.0   
2156 2021-08-14 22:30:00                              62.86            -0.0   
2157 2021-08-14 23:00:00                              32.26            -0.0   
2158 2021-08-14 23:30:00                              25.10            -0.0   
2159 2021-

## User Usage

In [ ]:
input_filepath = '' # Add filepath here (include extension)
output_filepath = '' # Add filepath here (include extension)

### IF INPUT IS EXCEL FILE EXTENSION

In [ ]:
dataframe = pd.read_excel(input_filepath)
dataframe = dataframe[["Time (UTC+10)", "Regions VIC Trading Price ($/MWh)"]] # Select time and victoria prices
dataframe.columns = ["Time", "Price"] # Rename columns
dataframe["Time"] = pd.to_datetime(dataframe["Time"]) # Convert data type
dataframe = dataframe.sort_values("Time").reset_index(drop = True) # Finalise
result = SandwichModel(dataframe)
print(f"Total Revenue for this dataset is : {pm.computeRevenue(result)}")

### IF INPUT IS CSV FILE EXTENSTION

In [ ]:
dataframe = pd.read_csv(input_filepath)
dataframe = dataframe[["Time (UTC+10)", "Regions VIC Trading Price ($/MWh)"]] # Select time and victoria prices
dataframe.columns = ["Time", "Price"] # Rename columns
dataframe["Time"] = pd.to_datetime(dataframe["Time"]) # Convert data type
dataframe = dataframe.sort_values("Time").reset_index(drop = True) # Finalise
result = SandwichModel(dataframe)
print(f"Total Revenue for this dataset is : {pm.computeRevenue(result)}")

### SAVE TO EXCEL

In [ ]:
result.to_excel(output_filepath, index = False)

### SAVE TO CSV

In [ ]:
result.to_csv(output_filepath, index = False)